In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import collections
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor, tensor
import pandas as pd
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from preprocessing.stream_transforms import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from utils.utils import print_name, print_shape
from models import ResNet, NeuralEulerODE, RidgeClassifierCVModule, E2EResNet, LogisticRegression, RandFeatBoost, GradientRandomFeatureBoostingClassification

np.set_printoptions(precision=3, threshold=5) # Print options
device = "cuda" #torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load CIFAR-10 Dataset

In [2]:
def extract_features(data_loader, model, device):
    """Function to extract features from a dataset using pre-train model"""
    features = []
    labels = []
    with torch.no_grad():
        for images, target in tqdm(data_loader):
            images = images.to(device)
            output = model(images)
            output = output.view(output.size(0), -1)
            features.append(output.cpu().numpy())
            labels.append(target.cpu().numpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels


def load_cifar10(
        data_path = "/home/nikita/hdd/cifar10/",
        train_name = "resnet18_train_features.csv",
        test_name = "resnet18_test_features.csv",
        ):
    """Loads a pretrained ResNet18 model and extracts features from the CIFAR-10 dataset"""
    # see if data has already been processed
    if train_name in os.listdir(data_path) and \
            test_name in os.listdir(data_path):
        print("Loading preprocessed data")
        train_df = pd.read_csv(data_path + train_name)
        test_df = pd.read_csv(data_path + test_name)
        return train_df.to_numpy(), test_df.to_numpy()

    # Define the DataLoaders and transformations for the CIFAR-10 dataset
    transform = transforms.Compose([
        transforms.Resize(224),  # Resize images to 224x224 as required by ResNet18
        transforms.ToTensor(),  # Convert images to PyTorch tensors
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize with ImageNet mean and std
    ])
    train_dataset = torchvision.datasets.CIFAR10(root=data_path, train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.CIFAR10(root=data_path, train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

    # Load the pre-trained ResNet18 model, remove classification head
    model = resnet18(weights=True)
    model = torch.nn.Sequential(*list(model.children())[:-1])
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Extract features for the training and test datasets
    train_features, train_labels = extract_features(train_loader, model, device)
    test_features, test_labels = extract_features(test_loader, model, device)

    # Create a DataFrame to store the features and labels, save to CSV
    train_df = pd.DataFrame(train_features)
    train_df['target'] = train_labels
    test_df = pd.DataFrame(test_features)
    test_df['target'] = test_labels
    train_df.to_csv(data_path + train_name, index=False)
    test_df.to_csv(data_path + test_name, index=False)
    return train_df.to_numpy(), test_df.to_numpy()

In [3]:
train, test = load_cifar10()

X_train = torch.from_numpy(train[:, :-1].astype(np.float32)).to(device)
y_train_cat = torch.from_numpy(train[:, -1].astype(np.int64)).to(device)
X_test = torch.from_numpy(test[:, :-1].astype(np.float32)).to(device)
y_test_cat = torch.from_numpy(test[:, -1].astype(np.int64)).to(device)

# Convert train and test labels to one-hot encoding
y_train = F.one_hot(y_train_cat, num_classes=10).float()
y_test = F.one_hot(y_test_cat, num_classes=10).float()
print(f"Train data shape: {X_train.shape}, dtype: {X_train.dtype}")
print(f"Train labels shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"Test data shape: {X_test.shape}, dtype: {X_test.dtype}")
print(f"Test labels shape: {y_test.shape}, dtype: {y_test.dtype}")

Loading preprocessed data
Train data shape: torch.Size([50000, 512]), dtype: torch.float32
Train labels shape: torch.Size([50000, 10]), dtype: torch.float32
Test data shape: torch.Size([10000, 512]), dtype: torch.float32
Test labels shape: torch.Size([10000, 10]), dtype: torch.float32


# experiments

In [17]:
def run_allmodels_1dataset(
        generator: torch.Generator,
        X_train: Tensor,
        y_train: Tensor,
        X_test: Tensor,
        y_test: Tensor,
        ):
    
    D = X_train.shape[1]
    hidden_size = 512
    bottleneck_dim = 1*hidden_size
    num_epochs = 40
    batch_size = 512
    adam_lr = 0.01
    
    # (name, model, kwargs). kwargs separate to save memory
    model_list = [
        # ["T=10 RandFeatureBoost", RandFeatBoost,
        #         {"generator": generator,
        #          "in_dim": D,
        #          "hidden_size": hidden_size,
        #          "out_dim": 10,
        #          "n_blocks": 9,
        #          "activation": nn.Tanh(),
        #          "loss_fn": nn.CrossEntropyLoss(),
        #          "adam_lr": adam_lr,
        #          "boost_lr": 1.0,
        #          "epochs": num_epochs,
        #          "batch_size": batch_size,
        #          "upscale_type": "SWIM",  # "dense", "identity"
        #          }],

        # ["Tabular RidgeClassifier", RidgeClassifierCVModule, {}],

    #     ["T=1 Dense", ResNet,
    #             {"generator": generator,
    #              "in_dim": D,
    #              "hidden_size": hidden_size,
    #              "bottleneck_dim": None,
    #              "n_blocks": 0,
    #              "upsample_layer": "dense",
    #              "output_layer": "logistic regression",
    #              }],

    #     ["T=1 SWIM Grad", ResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": None,
    #             "n_blocks": 0,
    #             "upsample_layer": "SWIM",
    #             "output_layer": "logistic regression",
    #             }],
    ]

    # for n_blocks in [3]:
    #     model_list += [
    #     [f"T={n_blocks+1} End2End", E2EResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": bottleneck_dim,
    #             "out_dim": 10,
    #             "n_blocks": n_blocks,
    #             "activation": nn.Tanh(),
    #             "loss": nn.CrossEntropyLoss(),
    #             "lr": adam_lr,
    #             "epochs": num_epochs,
    #             "batch_size": batch_size}
    #             ],

    #     [f"T={n_blocks+1} ResSWIM Grad-dense", ResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": bottleneck_dim,
    #             "n_blocks": n_blocks,
    #             "upsample_layer": "SWIM",
    #             "res_layer1": "SWIM",
    #             "res_layer2": "dense",
    #             "output_layer": "logistic regression",
    #             }
    #             ],

    #     [f"T={n_blocks+1} ResSWIM Grad-id", ResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": hidden_size,
    #             "n_blocks": n_blocks,
    #             "upsample_layer": "SWIM",
    #             "res_layer1": "SWIM",
    #             "res_layer2": "identity",
    #             "output_layer": "logistic regression",
    #             }
    #             ],

    #     [f"T={n_blocks+1} ResDense", ResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": hidden_size,
    #             "n_blocks": n_blocks,
    #             "upsample_layer": "dense",
    #             "res_layer1": "dense",
    #             "res_layer2": "identity",
    #             "output_layer": "logistic regression",
    #             }
    #             ],
    # ]
        
    for l2_reg in [ 0.00001]:
        model_list += [
            [f"Logistic L-BFSG, l2={l2_reg}", 
                LogisticRegression, 
                {"generator": generator,
                "in_dim": D,
                "out_dim": 10,
                "l2_reg": l2_reg,
                "lr": 1.0,
                "max_iter": 100,
                }],
        ]
    
        
    for n_blocks in range(0, 10 + 1, 5):
        model_list += [
            [f"T={n_blocks} Gradient GRFBoost", 
             GradientRandomFeatureBoostingClassification,
                {"generator": generator,
                "hidden_dim": hidden_size,
                "bottleneck_dim": bottleneck_dim,
                "out_dim": 10,
                "n_layers": n_blocks,
                "activation": nn.Tanh(),
                "l2_reg": 0.0001,
                "feature_type": "SWIM",
                "boost_lr": 0.5,
                "upscale": None,
                },
                ],
        ]
    
    results = []
    model_names = []
    for name, model, model_args in model_list:
        print(name)
        torch.cuda.empty_cache()
        t0 = time.perf_counter()
        model = model(**model_args).to(X_train.device)
        pred_train, _ = model.fit(X_train, y_train)
        t1 = time.perf_counter()
        pred_test = model(X_test)
        t2 = time.perf_counter()
        
        #convert to class predictions:
        if len(pred_train.shape) == 2:
            pred_train = torch.argmax(pred_train, dim=1)
            pred_test = torch.argmax(pred_test, dim=1)
        acc_train = (pred_train == y_train_cat).float().mean().item()
        acc_test = (pred_test == y_test_cat).float().mean().item()

        result = np.array( [acc_train, acc_test, t1-t0, t2-t1] )
        results.append( result )
        model_names.append( name )

    return model_names, results



def run_all_experiments(
        name_save: str = "PLACEHOLDER",
        ):
    # Fetch and process each dataset
    experiments = {}
    generator = torch.Generator(device=device).manual_seed(999)
    results = run_allmodels_1dataset(
        generator, X_train, y_train, X_test, y_test, 
        )
    experiments["MNIST"] = results

    # Save results
    # Assuming experiments is a dict where keys are dataset names and values are tuples (model_names, results)
    attributes = ["acc_train", "acc_test", "t_fit", "t_feat"]
    data_list = []
    # Process the data
    for dataset_name, (model_names, results) in experiments.items():
        dataset_data = {}
        for attr_idx, attribute in enumerate(attributes):
            for model_idx, model_name in enumerate(model_names):
                dataset_data[(attribute, model_name)] = results[model_idx][attr_idx]
        data_list.append(pd.DataFrame(dataset_data, index=[dataset_name]))

    # Combine all datasets into a single DataFrame
    df = pd.concat(data_list)
    df = df.sort_index(axis=1)
    df.to_pickle(f"cifar10_{name_save}.pkl")
    return df

In [18]:
_ = run_all_experiments()

Logistic L-BFSG, l2=1e-05
T=0 Gradient GRFBoost
T=5 Gradient GRFBoost
alpha 10.0
linesearch loss tensor(0.2803, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2803, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2781, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2781, device='cuda:0', grad_fn=<NllLossBackward0>)
Linesearch 3.584036350250244
Gradient hat norm tensor(6.7225, device='cuda:0')
alpha 0.02154434658586979
linesearch loss tensor(0.4472, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.4467, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.3316, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.3288, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.3286, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.3286, device='cuda:0', grad_fn=<NllLossBackward0>)
Linesearch 10.734461784362793
Gradient hat norm tensor

In [21]:
df = pd.read_pickle("cifar10_PLACEHOLDER.pkl")
df["acc_test"].mean().sort_values(ascending=False)

T=0 Gradient GRFBoost        0.8739
Logistic L-BFSG, l2=1e-05    0.8738
T=10 Gradient GRFBoost       0.8720
T=5 Gradient GRFBoost        0.8694
dtype: float64

In [7]:
# T=10 Gradient GRFBoost               0.8757
# T=5 Gradient GRFBoost                0.8743
# T=0 Gradient GRFBoost                0.8737
# T=15 Gradient GRFBoost               0.8737
# T=4 End2End                          0.8663
# Logistic L-BFSG, l2=0.001 lr=1.0     0.8309
# Logistic L-BFSG, l2=1e-05 lr=1.0     0.8304
# Logistic L-BFSG, l2=1e-06 lr=1.0     0.8303
# Logistic L-BFSG, l2=0.1 lr=1.0       0.8295
# Logistic L-BFSG, l2=0.0001 lr=1.0    0.8285
# Logistic L-BFSG, l2=0.01 lr=1.0      0.8221
# Logistic L-BFSG, l2=1 lr=1.0         0.7912
# dtype: float64

In [26]:
df["acc_train"].mean().sort_values(ascending=False)

Logistic L-BFSG, l2=1e-05    0.90666
T=0 Gradient GRFBoost        0.90484
T=10 Gradient GRFBoost       0.89434
T=5 Gradient GRFBoost        0.88914
dtype: float64

In [9]:
# For T residual blocks, plot the test accuracy

# Optuna